## 이진분류(기출3회)

In [17]:
# 1. 문제정의
# 평가: roc-auc
# target: TravelInsurance
# 최종파일: result.csv(컬럼 1개 pred, 1확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("3_train.csv")
test = pd.read_csv("3_test.csv")

#train.shape,test.shape
#print(train.head()) #TravelInsurance = target
#print(test.head())

print(train.isnull().sum())
print(test.isnull().sum())

print(train.info())
print(test.info())

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64
Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 


In [18]:
train = pd.read_csv("3_train.csv")
test = pd.read_csv("3_test.csv")

# 4. 데이터 전처리

from sklearn.preprocessing import LabelEncoder
target = train.pop('TravelInsurance')
cols = train.select_dtypes(include = 'object').columns
print(cols)
for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

#print(train)
#print(test)

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.15, random_state = 2024)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 500, max_depth = 7, random_state = 2024)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)
print(pred)

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val, pred[:,1])) # 기본 : 0.73348714004956

import xgboost as xgb
xg = xgb.XGBClassifier()
xg.fit(X_tr, y_tr)
xgPred = xg.predict_proba(X_val)
print(xgPred)
print(roc_auc_score(y_val, xgPred[:,1]))

testPred = rf.predict_proba(test)
#print(testPred) 0.7274630436640177
# 랜포 채택

submit = pd.DataFrame({
    'pred' : testPred
})

print(submit.info())
#submit.to_csv('result.csv', index = False)
# roc_auc score를 묻는 경우는 predict_proba임.
# 그 외 나머지의 경우는 predict

Index(['Employment Type', 'GraduateOrNot', 'FrequentFlyer',
       'EverTravelledAbroad'],
      dtype='object')
(1266, 9) (224, 9) (1266,) (224,)
[[0.62986067 0.37013933]
 [0.85529844 0.14470156]
 [0.79541099 0.20458901]
 [0.89523762 0.10476238]
 [0.85027205 0.14972795]
 [0.6908189  0.3091811 ]
 [0.24379826 0.75620174]
 [0.8172544  0.1827456 ]
 [0.8386063  0.1613937 ]
 [0.01614584 0.98385416]
 [0.0387615  0.9612385 ]
 [0.33546791 0.66453209]
 [0.74912377 0.25087623]
 [0.02678544 0.97321456]
 [0.79341874 0.20658126]
 [0.82289135 0.17710865]
 [0.0495247  0.9504753 ]
 [0.83885385 0.16114615]
 [0.02172478 0.97827522]
 [0.33038876 0.66961124]
 [0.79797369 0.20202631]
 [0.87989383 0.12010617]
 [0.73830008 0.26169992]
 [0.78473405 0.21526595]
 [0.80291268 0.19708732]
 [0.3079283  0.6920717 ]
 [0.84141816 0.15858184]
 [0.80611044 0.19388956]
 [0.77541708 0.22458292]
 [0.8418527  0.1581473 ]
 [0.81673653 0.18326347]
 [0.68141864 0.31858136]
 [0.16317926 0.83682074]
 [0.80514037 0.19485963]
 [0

ValueError: Per-column arrays must each be 1-dimensional

## 다중분류(기출4회)


In [19]:
# 1. 문제정의
# 평가: f1 macro
# target: Segmentation
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("4_train.csv")
test = pd.read_csv("4_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape) #최종으로 제출할 test의 길이와 test.shape를 통해 얻은 데이터의 길이가 동일해야 함!
print(train.head())
print(test.head())
print("===== 데이터 정보(자료형) =====")
print(train.info())
print("\n")

print("===== train 결측치 수 =====")
print(train.isnull().sum().sum())
print("\n")

print("===== test 결측치 수 =====")
print(test.isnull().sum().sum())
print("\n")

print("===== target 빈도 =====")
print(train['Segmentation'].value_counts())



===== 데이터 크기 =====
Train Shape: (6665, 11)
Test Shape: (2154, 10)
       ID  Gender Ever_Married  Age Graduated  Profession  Work_Experience  \
0  462809    Male           No   22        No  Healthcare              1.0   
1  466315  Female          Yes   67       Yes    Engineer              1.0   
2  461735    Male          Yes   67       Yes      Lawyer              0.0   
3  461319    Male          Yes   56        No      Artist              0.0   
4  460156    Male           No   32       Yes  Healthcare              1.0   

  Spending_Score  Family_Size  Var_1  Segmentation  
0            Low          4.0  Cat_4             4  
1            Low          1.0  Cat_6             2  
2           High          2.0  Cat_6             2  
3        Average          2.0  Cat_6             3  
4            Low          3.0  Cat_6             3  
       ID  Gender Ever_Married  Age Graduated  Profession  Work_Experience  \
0  458989  Female          Yes   36       Yes    Engineer            

In [20]:

train = pd.read_csv("4_train.csv")
test = pd.read_csv("4_test.csv")

target = train.pop("Segmentation")
print(target)

cols = train.select_dtypes(include = 'object').columns
print(cols)

from sklearn.preprocessing import LabelEncoder

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])


from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.15, random_state = 2024)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 400, max_depth = 9, random_state = 2024)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

#print(pred)

testPred = rf.predict(test)

# print(testPred)

from sklearn.metrics import f1_score
print(f1_score(y_val, pred, average = 'macro')) #0.5261950474189155
#0.5461805614282873 하이퍼파러미터 적용


submit = pd.DataFrame({
    'pred' : testPred
})

print(submit.info())

#submit.to_csv('result.csv', index=False)


import lightgbm as lgb
lg = lgb.LGBMClassifier()
lg.fit(X_tr, y_tr)
lgPred = lg.predict(X_val)
print(lgPred)
print(f1_score(y_val, lgPred, average= 'macro')) #0.5461805614282873

# 랜덤포레스트 하이퍼패러미터로 채택.

0       4
1       2
2       2
3       3
4       3
       ..
6660    2
6661    4
6662    4
6663    2
6664    2
Name: Segmentation, Length: 6665, dtype: int64
Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object')
(5665, 10) (1000, 10) (5665,) (1000,)
0.5461805614282873
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   pred    2154 non-null   int64
dtypes: int64(1)
memory usage: 17.0 KB
None
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 5665, number of used features: 10
[LightGBM] [Info] Start training from score -1.423152
[LightGBM

## 회귀(기출5회)

In [21]:
# 1. 문제정의
# 평가: RMSE
# target: price
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("5_train.csv")
test = pd.read_csv("5_test.csv") #1617행. 제출하는 submit의 길이도 1617인지 확인!
# train에 price가 존재. 이를 test에 검증.
# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum()) #결측치 없음.

print("\n ===== 카테고리 비교 =====")
print(train.describe(include='O'))
print(test.describe(include='O'))

# 각 컬럼별 카테고리 확인 및 비교 (3개중 1개만 샘플)
print(train['fuelType'].value_counts())
print(test['fuelType'].value_counts())

print("\n ===== target 기술 통계 =====")
print(train['price'].describe())

===== 데이터 크기 =====
Train Shape: (3759, 9)
Test Shape: (1617, 8)

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3759 entries, 0 to 3758
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3759 non-null   object 
 1   year          3759 non-null   int64  
 2   price         3759 non-null   int64  
 3   transmission  3759 non-null   object 
 4   mileage       3759 non-null   int64  
 5   fuelType      3759 non-null   object 
 6   tax           3759 non-null   int64  
 7   mpg           3759 non-null   float64
 8   engineSize    3759 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 264.4+ KB
None

 ===== train 결측치 수 =====
0

 ===== test 결측치 수 =====
0

 ===== 카테고리 비교 =====
          model transmission fuelType
count      3759         3759     3759
unique       19            3        3
top      Fiesta       Manual   Petrol
freq       1372         3234     

### 응용

In [22]:
print("\n ===== 카테고리 비교 =====")

cols = train.select_dtypes(include = 'object').columns
print(cols)

for col in cols:
  if set(train[col]) == set(test[col]):
    print(col, '컬럼 동일')
  else:
    print(col, '컬럼 다름')

# 범주형의 컬럼은 train과 test 모두 동일.


 ===== 카테고리 비교 =====
Index(['model', 'transmission', 'fuelType'], dtype='object')
model 컬럼 동일
transmission 컬럼 동일
fuelType 컬럼 동일


In [35]:
train = pd.read_csv("5_train.csv")
test = pd.read_csv("5_test.csv")

# price 컬럼을 분리
target = train.pop('price')
#print(target)

print(train.shape, test.shape)

train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.15, random_state = 2024)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 2024)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print(pred)

#검증 데이터는 rmse

from sklearn.metrics import mean_squared_error

# print(help(mean_squared_error))

def rmse(a,b):
  return mean_squared_error(a,b) ** 0.5

print(rmse(y_val, pred))
 #베이스라인은 : 1195.0867995604576
 # 하이퍼 파라미터 적용 후: 1451.3516654963698
 # error값은 낮은 것이 이득이므로 기본으로 채택

test_pred = rf.predict(test)
submit = pd.DataFrame({
    'pred': test_pred
})

print(submit.shape, submit.head())
submit.to_csv('result.csv', index = False)

(3759, 8) (1617, 8)
(3759, 30) (1617, 30)
(3195, 30) (564, 30) (3195,) (564,)
[ 8292.65       16616.02       11431.86       10043.41
 13967.98        9043.59       11107.77       11931.73
 11156.5         9810.8        16520.09       11500.7
  6919.39       10670.01        5043.46       14501.19
  7704.42       17755.21       11690.36       16690.4
 12142.05       10257.93        7080.69       19207.83
 12041.09       10890.29       17080.38        3952.45
  8858.44        6203.53       10028.61       11059.41
  8584.91       10691.28       14956.67        9360.11
 13724.78       18218.46       12095.45       11320.88
 10459.03       18938.04       13919.75       20301.725
  6760.95       12117.35       11960.28816667 16584.42
  8337.41       15714.84       12372.75       10728.82
 16519.92        8984.81       14996.27       14069.34
  3727.3        24864.5        19611.87        9841.78
 18318.16        9430.09       11011.87        9981.66
  9789.95        9167.75        9509.75    

## 다중분류(기출6회)

In [36]:
# 1. 문제정의
# 평가: f1 macro
# target: Heat_Load
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("6_train.csv")
test = pd.read_csv("6_test.csv") # test 길이는 231, target은 Heat_Load

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
print("\n") # 줄 바꿈

print("===== 데이터 정보(자료형) =====")
print(train.info())
print("\n")

print("===== train 결측치 수 =====")
print(train.isnull().sum().sum())
print("\n")

print("===== test 결측치 수 =====")
print(test.isnull().sum().sum())
print("\n")

print("===== target 빈도 =====")
print(train['Heat_Load'].value_counts())

===== 데이터 크기 =====
Train Shape: (537, 10)
Test Shape: (231, 9)


===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       537 non-null    float64
 1   Surf_Area    537 non-null    float64
 2   Wall_Area    537 non-null    float64
 3   Roof         537 non-null    object 
 4   Height       537 non-null    object 
 5   Orient       537 non-null    object 
 6   Glaze_Area   537 non-null    float64
 7   Glaze_Distr  537 non-null    int64  
 8   Cool_Load    537 non-null    float64
 9   Heat_Load    537 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 42.1+ KB
None


===== train 결측치 수 =====
0


===== test 결측치 수 =====
0


===== target 빈도 =====
Heat_Load
Very Low     142
Low          123
High         122
Very High     79
Medium        71
Name: count, dtype: int64


In [69]:
train = pd.read_csv("6_train.csv")
test = pd.read_csv("6_test.csv")

# 4. 데이터 전처리

target = train.pop('Heat_Load')
print(target)

# 각 데이터를 인코딩 개시. 이번의 경우는 label인코딩 사용.

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
cols = train.select_dtypes(include = 'object').columns

for col in cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

print(train.shape, test.shape)

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.1, random_state = 2024)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 600, max_depth = 9)
rf.fit(X_tr, y_tr)
rfPred = rf.predict(X_val)
#print(rfPred)

from sklearn.metrics import f1_score

#print(f1_score(y_val, pred, average = 'macro'))
#print(f1_score(y_val, rfPred, average = 'macro')) #0.9156284565108095 베이스라인

rfPred = rf.predict(test)
submit = pd.DataFrame({
    'pred' : rfPred
})

submit.to_csv('result.csv', index=False)

0       Very Low
1           High
2           High
3       Very Low
4           High
         ...    
532     Very Low
533         High
534    Very High
535    Very High
536         High
Name: Heat_Load, Length: 537, dtype: object
(537, 9) (231, 9)
(483, 9) (54, 9) (483,) (54,)


## 회귀(기출7회)

In [70]:
# 1. 문제정의
# 평가: RMSE
# target: total
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("7_train.csv")
test = pd.read_csv("7_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum())

print("\n ===== 카테고리 비교 =====")
print(train.describe(include='O'))
print(test.describe(include='O'))

print("\n ===== target 기술 통계 =====")
print(train['total'].describe())

===== 데이터 크기 =====
Train Shape: (700, 10)
Test Shape: (300, 9)

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          700 non-null    object 
 1   city            700 non-null    object 
 2   customer_type   700 non-null    object 
 3   gender          700 non-null    object 
 4   product_line    700 non-null    object 
 5   total           700 non-null    float64
 6   payment_method  700 non-null    object 
 7   rating          700 non-null    float64
 8   time_of_day     700 non-null    object 
 9   day_name        700 non-null    object 
dtypes: float64(2), object(8)
memory usage: 54.8+ KB
None

 ===== train 결측치 수 =====
0

 ===== test 결측치 수 =====
0

 ===== 카테고리 비교 =====
       branch    city customer_type gender       product_line payment_method  \
count     700     700           700    700         

In [ ]:
pd.set_option('display.float_format', '{:.10f}'.format)
train['total'].describe()

count       700.0000000000
mean     485078.0175000000
std      364390.7265411940
min       19041.7500000000
25%      200119.5000000000
50%      381874.5000000000
75%      706127.6250000000
max     1563975.0000000002
Name: total, dtype: float64

In [80]:
train = pd.read_csv("7_train.csv")
test = pd.read_csv("7_test.csv")

target = train.pop('total')
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(test.shape)
#print(train.shape, test.shape)

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size =0.1, random_state = 2024)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth = 3)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import mean_squared_error

def rmse (a,b):
  return (mean_squared_error(a,b) ** 0.5)

print(rmse(y_val, pred))


testPred = rf.predict(test)

submit = pd.DataFrame({
    'pred': testPred
})

print(submit.shape)

submit.to_csv('result.csv', index = False)

new_data = pd.read_csv('result.csv')
print(new_data)

(300, 30)
(630, 30) (70, 30) (630,) (70,)
364467.4895057684
(300, 1)
              pred
0    483675.446639
1    569942.101517
2    430948.997983
3    497683.102778
4    481679.447962
..             ...
295  460186.806727
296  525515.360411
297  493642.733773
298  536926.335159
299  475033.623451

[300 rows x 1 columns]
